主要功能：基于genecompass需要的数据预处理工作，调整H5AD文件，并补充对应基因编码对应索引。生成新的H5AD文件用作GEARS的输入

In [ ]:
import scanpy as sc
import numpy as np

aData = sc.read_h5ad('/ssd/home/ict00/norman_geneformer_pre/perturb_processed.h5ad')
print(aData)

link=list(aData.var['gene_name'].index)



from datasets import load_from_disk
import pickle

with open('/ssd/home/ict00/0409_work/cell.pickle','rb') as f:
    data=pickle.load(f)

choose=[]


for i in data:
    if i==True:
        choose.append(0)
    else:
        choose.append(1)

print(data)
print(sum(choose))





with open('/ssd/home/ict00/0409_work/gene.pkl','rb') as f:
    gene_sort=pickle.load(f)

gene_token_path = '/data/home/ict01/geneformer/geneformer/token_dictionary.pkl' #token路径，需要改

with open(gene_token_path,'rb') as f:
    gene_dic=pickle.load(f)


my_dic={value: key for key, value in gene_dic.items()}

gene_sort=np.array(gene_sort)

jihe=set()
for i in range(len(gene_sort)):
    for j in range(2048):
        jihe.add(gene_sort[i][j])
    break

print(jihe)
print(len(jihe))


yzh_layers=[]
for i in range(len(gene_sort)):
    mid=[]
    last=0
    for j in range(3248):
        mid.append(-1)
    for j in range(2048):
        if gene_sort[i][j]!=0:
            # print(my_dic[gene_sort[i][j]])
            # print(link.index(my_dic[gene_sort[i][j]]))

            mid[link.index(my_dic[gene_sort[i][j]])]=j
        else:
            last=j
            break
    for j in range(3248):
        if mid[j]==-1:
            mid[j]=last
    yzh_layers.append(mid)

# print(yzh_layers)

yzh_layers=np.array(yzh_layers)
print(yzh_layers.shape)

# 这里就是一个对应索引的层
with open('my_gene_sort_layers.pickle','wb') as f:
    pickle.dump(yzh_layers,f)

这里就是基于前面增加的部分，重新构建一个H5AD用作输入

In [ ]:
import scanpy as sc
import pickle

with open('my_gene_sort_layers.pickle','rb') as f:
    data=pickle.load(f)

print(data)

aData = sc.read_h5ad('perturb_processed.h5ad')

print(aData)

aData.obs['flag'] = range(len(aData.obs))

aData.layers['new_layers']=data

print(aData)

aData.write('yzh_file.h5ad')

print(aData.obs['flag'])